# -*- coding: utf-8 -*-
"""
Created on Mon Mar 26 2022
@author: Amirmohammad Shahbandegan
"""
#

In [ ]:
!pip install emoji nltk spacy contractions sentence_transformers

     |████████████████████████████████| 175 kB 5.6 MB/s 
     |████████████████████████████████| 79 kB 7.0 MB/s 
     |████████████████████████████████| 106 kB 33.1 MB/s 
     |████████████████████████████████| 284 kB 36.9 MB/s 
     |████████████████████████████████| 3.8 MB 41.4 MB/s 
     |████████████████████████████████| 1.2 MB 33.7 MB/s 
     |████████████████████████████████| 67 kB 2.7 MB/s 
     |████████████████████████████████| 895 kB 33.6 MB/s 
     |████████████████████████████████| 596 kB 4.2 MB/s 
     |████████████████████████████████| 6.5 MB 9.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=dccff71f524d6ebb25a28b0e83d14b599d4867a43371640c516eef9259e9d18f
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=01f6cc1e87e5770d5ebfe11b90f6874a93049837d4f45c9743c2d3d

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!wget https://github.com/am-shb/cyberbullying-detection/archive/refs/heads/main.zip
!unzip main.zip
!mv cyberbullying-detection-main/src/* .
!mv cyberbullying-detection-main/data/* .

--2022-04-01 14:41:27--  https://github.com/am-shb/cyberbullying-detection/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/am-shb/cyberbullying-detection/zip/refs/heads/main [following]
--2022-04-01 14:41:27--  https://codeload.github.com/am-shb/cyberbullying-detection/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]   2.79M  14.6MB/s    in 0.2s    

2022-04-01 14:41:28 (14.6 MB/s) - ‘main.zip’ saved [2921649]

Archive:  main.zip
fe5462c9435bb19ee323b45f631ca89e77f39b9e
   creating: cyberbullying-detection-main/
 extracting:

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from preprocessing import TweetProcessor

tqdm.pandas()

tp1 = TweetProcessor(lowercase=True, remove_stopwords=True, lemmatize=True,
                    replace_urls='', replace_mentions='', replace_hashtags=False,
                    convert_to_ascii=False, remove_punctuation=True,
                    remove_numbers=True, replace_emojis=True,
                    expand_contractions=True)


tp2 = TweetProcessor(lowercase=True, remove_stopwords=True, lemmatize=False,
                    replace_urls='', replace_mentions='', replace_hashtags=False,
                    convert_to_ascii=False, remove_punctuation=True,
                    remove_numbers=True, replace_emojis=True,
                    expand_contractions=True)

df = pd.read_csv('cyberbullying_tweets.csv')

tweets1 = df['tweet_text'].progress_apply(tp1.transform).to_numpy()
tweets2 = df['tweet_text'].progress_apply(tp2.transform).to_numpy()
labels = df['cyberbullying_type'].to_numpy()
binary_labels = ~(df['cyberbullying_type'] == 'not_cyberbullying').to_numpy()

100%|██████████| 47692/47692 [00:08<00:00, 5741.96it/s]


In [ ]:
from embeddings import FrequencyVectorizer, Word2vecVectorizer, SbertVectorizer
embedders1 = {
    'bow': FrequencyVectorizer(kind='bow', max_features=10000, ngram_range=(1, 5)),
    'tfidf': FrequencyVectorizer(kind='tfidf', max_features=10000, ngram_range=(1, 5))
}

embedders2 = {
    'w2v': Word2vecVectorizer('word2vec-google-news-300'),
    'glove': Word2vecVectorizer('glove-wiki-gigaword-300'),
    'ft': Word2vecVectorizer('fasttext-wiki-news-subwords-300'),
    'sbert': SbertVectorizer('all-mpnet-base-v2')
}

embedders1['bow'].fit(tweets1)
embedders1['tfidf'].fit(tweets1)

In [ ]:
embeddings = {}
for name, embedder in embedders1.items():
    embeddings[name] = embedders1[name].batch_encode(tweets1)

for name, embedder in embedders2.items():
    embeddings[name] = embedders2[name].batch_encode(tweets2)

/content/embeddings/Word2vecVectorizer.py:22: RuntimeWarning: Mean of empty slice.
  encoded = np.array([self.model[token] for token in tokenize(text) if token in self.model]).mean(axis=0)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
import pickle
with open('embeddings.pkl', 'wb+') as f:
  pickle.dump(embeddings, f)